if input1 == "Menos de 15.000km" and input2 == "Sobretodo por ciudad y muy pocos viajes por carretera al año." and input3 == "Suelo viajar sólo y/o una persona más.":
    Tipo1
elif input1 == "Menos de 15.000 km" and input2 == "Por ciudad y carretera por igual." and input3 == "Suelo viajar sólo y/o una persona más.":
    Tipo2
elif input1 == "Más de 15.000 km" and input2 == "Uso el vehículo sobretodo para viajar y muy poco por ciudad." and input3 == "Suelo viajar sólo y/o una persona más.":
    Tipo3
elif input1 == "Más de 15.000 km" and input2 == "Uso el vehículo sobretodo para viajar y muy poco por ciudad." and input3 == "Suelo viajar con la familia (3 personas máx).":
    Tipo4
elif input1 == "Menos de 15.000km" and input2 == "Sobretodo por ciudad y muy pocos viajes por carretera al año." and input3 == "Suelo viajar con la familia (3 personas máx).":
    Tipo5
elif input1 == "Menos de 15.000km" and input2 == "Sobretodo por ciudad y muy pocos viajes por carretera al año." and input3 =="Suelo viajar con la familia (más de 3 personas).":
    Tipo6
elif input1 == "Más de 15.000km" and input2 == "Uso el vehículo sobretodo para viajar y muy poco por ciudad." and input3 =="Suelo viajar con la familia (más de 3 personas).":
    Tipo7



In [2]:
import streamlit as st
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import streamlit as st
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import re

In [12]:
# Tipos de usuarios:

Tipo1="https://www.flexicar.es/combustible-gasolina/carroceria-compacto/segunda-mano/"
Tipo2= "https://www.flexicar.es/combustible-gasolina/carroceria-turismo/segunda-mano/"
Tipo3= "https://www.flexicar.es/combustible-diesel/carroceria-turismo/segunda-mano/"
Tipo4= "https://www.flexicar.es/combustible-diesel/carroceria-familiar/segunda-mano/"
Tipo5= "https://www.flexicar.es/combustible-gasolina/carroceria-familiar/segunda-mano/"
Tipo6= "https://www.flexicar.es/combustible-gasolina/carroceria-monovolumen/segunda-mano/"
Tipo7= "https://www.flexicar.es/combustible-diesel/carroceria-monovolumen/segunda-mano/"

In [13]:
def concat(*args):
        strs = [str(arg) for arg in args if not pd.isnull(arg)]
        return ','.join(strs) if strs else np.nan

In [14]:
def FlexicarScraping(tipo):
    #URL=f'https://www.flexicar.es/coches-segunda-mano/#precio_to/{presupuesto}/'
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("start-maximized")
    options.add_argument("--disable-infobars")
    driver = webdriver.Chrome(executable_path=r'/usr/bin/chromedriver',options=options) 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    driver.get(tipo)
    precio= driver.find_elements_by_xpath('//*[@id="listadocoches"]')
    cochesBuenos= [e.text for e in precio]
    for e in cochesBuenos:
        ok=e.split("\n")
    droplist=['Previous','Next',"IVA deducible"]
    cleanList=[e for e in ok if e not in droplist]
    CleanList=[]
    for e in cleanList:
        if not "desde" in e:
             CleanList.append(e)
    priceWithDiscount=CleanList[1::4]
    model=CleanList[2::4]
    cleanColumn=CleanList[3::4]
    for e in cleanColumn:
        ok=e.split("\n")
    cleanColumn2=[e.split("\n") for e in cleanColumn]
    ok2=[]
    for e in cleanColumn2:
        for i in e:
            ok2.append(i.split())
    Año=[e[0] for e in ok2]
    Combustible=[e[2] for e in ok2]
    combustiblesucio=[re.findall(r"([$D-H](\w+))",e) for e in Combustible]
    combustiblesucio=[list(item) for sublist in combustiblesucio for item in sublist]
    combustible=[item for sublist in combustiblesucio for item in sublist]
    gas=[]
    for e in combustible:
        if e == 'Hibrido':
            gas.append(e)
        elif e == "Diesel":
            gas.append(e)
        elif e == "Gasolina":
            gas.append(e)
        elif e == "Electrico":
            gas.append(e)
        elif e == "GLP":
            gas.append(e)
    df = pd.DataFrame({
                   'priceWithDiscount': priceWithDiscount,
                     'Model' : model,
                     'Combustible': gas,
                     'Año': Año,
                     'cleanToSplit': cleanColumn
                  })
    df1 = pd.DataFrame(df.cleanToSplit.str.split(' ',4).tolist(),
                                 columns = ["Año1","KM","3","Ciudad","nan"])
    result = pd.concat([df, df1], axis=1, sort=False)
    flexicar = result.drop(["cleanToSplit",'Año1','3'], axis=1)
    np_concat = np.vectorize(concat)
    flexicar['Ciudad'] = np_concat(flexicar['Ciudad'], flexicar['nan'])
    flexicar = flexicar.drop(['nan'], axis=1)
    
    
    return flexicar.sort_values(by='Año', ascending=[True])

In [15]:
km = st.multiselect(
    "¿Cuántos Km haces al año?", ["Más de 15.000 km","Menos de 15.000km"]
)
uso = st.multiselect(
    "¿Cómo serán tus desplazamientos con tu vehículo?",
     ["Sobretodo por ciudad y muy pocos viajes por carretera al año.",
     "Por ciudad y carretera por igual.", "Uso el vehículo sobretodo para viajar y muy poco por ciudad.",
     ]
)
familia = st.multiselect(
    "¿Cuántas personas suelen viajar contigo?",
     ["Suelo viajar sólo y/o una persona más.",
     "Suelo viajar con la familia (3 personas máx).", 
     "Suelo viajar con la familia (más de 3 personas)."
     ]
)

In [16]:
km

[]

In [8]:
if km[1] and uso[0]  and familia[0]:
    df=FlexicarScraping(Tipo1)
    st.dataframe(df)
elif km[1] and uso[1]  and familia[0]:
    df1=FlexicarScraping(Tipo2)
    st.dataframe(df1)
    print("ok")
elif km[0] and uso[2]  and familia[0]:
    df2=FlexicarScraping(Tipo3)
    st.dataframe(df2)
elif km[0] and uso[2] and familia[1]:
    df3=FlexicarScraping(Tipo4)
    st.dataframe(df3)
elif km[1] and uso[0] and familia[1]:
    df4=FlexicarScraping(Tipo5)
    st.dataframe(df4)
elif km[1] and uso[0] and familia[2]:
    df5=FlexicarScraping(Tipo6)
    st.dataframe(df5)
elif km[0] and uso[2] and familia[2]:
    df6=FlexicarScraping(Tipo7)
    st.dataframe(df6)

IndexError: list index out of range

In [ ]:
Tipo1="https://www.flexicar.es/combustible-gasolina/carroceria-compacto/segunda-mano/"
Tipo2= "https://www.flexicar.es/combustible-gasolina/carroceria-turismo/segunda-mano/"
Tipo3= "https://www.flexicar.es/combustible-diesel/carroceria-turismo/segunda-mano/"
Tipo4= "https://www.flexicar.es/combustible-diesel/carroceria-familiar/segunda-mano/"
Tipo5= "https://www.flexicar.es/combustible-gasolina/carroceria-familiar/segunda-mano/"
Tipo6= "https://www.flexicar.es/combustible-gasolina/carroceria-monovolumen/segunda-mano/"
Tipo7= "https://www.flexicar.es/combustible-diesel/carroceria-monovolumen/segunda-mano/"

In [ ]:
if km[1] and uso[0] and familia[0]:
    df=FlexicarScraping(Tipo1)
    st.dataframe(df)
elif km[1] and uso[1] and familia[0]:
    df1=FlexicarScraping(Tipo2)
    st.dataframe(df1)
    print("ok")
elif km[0] and uso[2] and familia[0]:
    df2=FlexicarScraping(Tipo3)
    st.dataframe(df2)
elif km[0] and uso[2] and familia[1]:
    df3=FlexicarScraping(Tipo4)
    st.dataframe(df3)
elif km[1] and uso[0] and familia[1]:
    df4=FlexicarScraping(Tipo5)
    st.dataframe(df4)
elif km[1] and uso[0] and familia[2]:
    df5=FlexicarScraping(Tipo6)
    st.dataframe(df5)
elif km[0] and uso[2] and familia[2]:
    df6=FlexicarScraping(Tipo7)
    st.dataframe(df6)
